In [12]:
import pandas as pd
import os

saveHere = r'./Data/GCFTables'

##inputs
bgcInfo = pd.read_csv(r'./Data/SupTable4.tsv', sep='\t')
bgcKey = pd.read_csv(r'./Data/GCFs_Keys.tsv', sep='\t')
bgcKey = bgcKey[["Cluster_Name","FinalGCFs"]]

merged = bgcInfo.merge(bgcKey, on="Cluster_Name",how="left")

for gcf in merged.groupby("FinalGCFs"):
    gcfName = gcf[0]
    #removing the * character if it's in the string
    if "*" in gcfName:
        gcfName = str.replace(gcfName, "*","_Refined")
    if " " in gcfName:
        gcfName = str.replace(gcfName, " ","_")
    gcfDF = gcf[1]
    savePath = os.path.join(saveHere, gcfName + ".tsv")
    gcfDF.to_csv(savePath,sep='\t',index=False)



In [4]:
bgcKey

,FinalGCFs,Cluster_Name,NCBI_Accession
0,1*,1_Talaromyces_marneffei,GCA_000001985.1
1,1*,4_Talaromyces_marneffei,GCA_000001985.1
2,1*,12_Talaromyces_stipitatus,GCA_000003125.1
3,1*,13_Talaromyces_stipitatus,GCA_000003125.1
4,1*,282_Talaromyces_marneffei,GCA_000750115.1
...,...,...,...
3505,209,3318_Clonostachys_rhizophaga,GCA_902077795.2
3506,209,3320_Clonostachys_rosea,GCA_902085965.1
3507,209,3322_Clonostachys_solani,GCA_902141235.2
3508,210,3356_Hymenoscyphus_fraxineus,GCA_911649665.1


In [5]:
bgcInfo

,Cluster_Name,NCBI_Accession,Protein,Domain,Domain_Accession,E-value_hmmer
0,1_Talaromyces_marneffei,GCA_000001985.1,EEA28616.1,Transglut_core,PF01841.20,1.600000e-12
1,1_Talaromyces_marneffei,GCA_000001985.1,EEA28617.1,Arf,PF00025.22,1.000000e-06
2,1_Talaromyces_marneffei,GCA_000001985.1,EEA28617.1,Ras,PF00071.23,9.300000e-31
3,1_Talaromyces_marneffei,GCA_000001985.1,EEA28617.1,Roc,PF08477.14,7.900000e-24
4,1_Talaromyces_marneffei,GCA_000001985.1,EEA28618.1,ANAPC4_WD40,PF12894.8,1.800000e-07
...,...,...,...,...,...,...
64504,3800_Alternaria_atra,GCF_907166805.1,XP_043172905.1,Transferase,PF02458.16,7.100000e-08
64505,3800_Alternaria_atra,GCF_907166805.1,XP_043172906.1,Abhydrolase_3,PF07859.14,1.400000e-44
64506,3800_Alternaria_atra,GCF_907166805.1,XP_043172906.1,COesterase,PF00135.29,5.900000e-10
64507,3800_Alternaria_atra,GCF_907166805.1,XP_043172907.1,AATase,PF07247.13,2.400000e-15


# Fix the gbk files listed in the website

In [23]:
import os
import pdb 
import shutil

gbkLocation = r'/Volumes/T7/ICSProject/ConsolidatedResults_3300Genomes/ICSGenomes/AllBioResults/AllBioSynGBKS'
pathLocation = r'/Users/gnickles/Desktop/FungalICS_Website/Data/ICSBGCs'

###Starter step, make a dictionary with all the gbk paths and make the names what they should be
# example of how the files are currently named: GCA_000001985.1_1_Talaromyces_marneffei
gbkDictionary = {}
for gbk in os.listdir(gbkLocation):
    gbkPath = os.path.join(gbkLocation, gbk)
    if os.path.isfile(gbkPath) and not gbk.startswith("."):
        clusterName = "_".join(gbk.split('_')[2:])
        clusterNumber = clusterName.split("_")[0]
        #
        newFilePath = os.path.join(gbkLocation,clusterName)
        os.rename(gbkPath, newFilePath)
        #adding this path to the dictionary 
        gbkDictionary[clusterNumber] = newFilePath

In [24]:
makeTheseArchives = []

##Looping over the folder and updating it with the correct files
counter = 0
for genome in os.listdir(pathLocation):
    genomePath = os.path.join(pathLocation, genome)
    # Step 0: get rid of the GCF folders, they're not needed and have the same info as the GCA folders
    if genome.startswith("GF") and os.path.isdir(genomePath):
        shutil.rmtree(genomePath)

    ### The main purpose of the script, swap out the files and make the zips again for brandon
    if os.path.isdir(genomePath) and genome.startswith("GC"):     
        ## Step 1: remove all of the zip files
        for item in os.listdir(genomePath):
            itemPath = os.path.join(genomePath, item)
            if os.path.isfile(itemPath) and item.endswith(".zip"):
                #remove that path
                os.remove(itemPath)
        ## Step 2: swap all of the gbk files for the new ones
        for item in os.listdir(genomePath):
            itemPath = os.path.join(genomePath, item)
            if os.path.isdir(itemPath) and item.startswith("Cluster"):
                clusterNumber = item.split("r_")[-1]
                #getting this path from the dictionary
                try:
                    clusterPath = gbkDictionary[clusterNumber]
                except:
                    continue #move onto the next cluster folder
                clusterName = clusterPath.split("/")[-1]
                moveHere = os.path.join(itemPath, clusterName)
                shutil.move(clusterPath, moveHere)
                ## Step 3: saving the path for zipping up the cluster folder later one
                makeTheseArchives.append(itemPath)
        ## Step 4: compressing the genome folder
        zipGenomePath = os.path.join(genomePath, genome)
        shutil.make_archive(zipGenomePath, "zip", genomePath)
        
        counter += 1
        print(str(counter) + " finished")

#Step 5: Making all of the zips
for itemPath in makeTheseArchives:
    shutil.make_archive(itemPath, "zip", itemPath)


1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
52 finished
53 finished
54 finished
55 finished
56 finished
57 finished
58 finished
59 finished
60 finished
61 finished
62 finished
63 finished
64 finished
65 finished
66 finished
67 finished
68 finished
69 finished
70 finished
71 finished
72 finished
73 finished
74 finished
75 finished
76 finished
77 finished
78 finished
79 finished
80 finished
81 finished
82 finished
83 finished
84 finished
8